In [2]:
import pandas as pd

In [3]:
df2 = pd.read_csv('test_processed.csv')

In [4]:
df1 = pd.read_csv('train_process2.csv')

In [5]:
df1.head()

,stars,text
0,1.0,purchase_jeep rubicon day still linger item ta...
1,4.0,stay best girlfriend husband place pretty supe...
2,1.0,rude disorganize incompetent jason bradley pas...
3,4.0,visit short trip_montreal friend recommend pla...
4,5.0,everything great nice price nice portion delis...


In [6]:
text = list(df1.text) + list(df2.text)

In [7]:
text = pd.DataFrame(text)
text.rename(columns = {0:'text'},inplace = True)

In [8]:
len(text)

2321274

construct TFIDF design matrix

In [9]:
from sklearn.feature_extraction.text import CountVectorizer  
vectorizer = CountVectorizer(max_df = 0.6)  
matrix = vectorizer.fit_transform(text['text'].values.astype('U'))

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer  
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(matrix)  

In [11]:
tfidf.shape

(2321274, 628798)

In [12]:
train = tfidf[:1000000]
test = tfidf[1000000:]

In [13]:
train.shape

(1000000, 628798)

use dev set to evaluate model performance

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, df1['stars'], test_size=0.3, random_state=123)

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from math import sqrt
classifier = LogisticRegression(C=2.2,penalty="l2",multi_class="multinomial",solver="saga")
classifier.fit(X_train,y_train)
y_pro = classifier.predict_proba(X_test)
pre = [sum(np.array([1,2,3,4,5])*i) for i in y_pro]
m = mean_squared_error(y_test,pre)
sqrt(m)

0.6572774704883365

Final model

In [16]:
classifier = LogisticRegression(C=2.2,penalty="l2",multi_class="multinomial",solver="saga")
classifier.fit(train,df1['stars'])
y_pro = classifier.predict_proba(test)
pre = [sum(np.array([1,2,3,4,5])*i) for i in y_pro]

model coeffieicents

In [35]:
feature_name = vectorizer.get_feature_names()

In [33]:
coef1 = classifier.coef_[0]
list(coef1).index(max(coef1))

169309

In [34]:
max(coef1)

10.942782735613829

In [36]:
feature_name[169309]

'give_zero'

In [37]:
coef5 = classifier.coef_[4]
list(coef5).index(max(coef5))

188112

In [38]:
max(coef5)

11.552010836315283

In [39]:
feature_name[188112]

'highly_recommend'

save predictions

In [ ]:
Id = range(1,len(pre)+1)
data = [Id,pre]
data = pd.DataFrame(data)
data.rename(columns = {0:'Id',1:'Expected'},inplace = True)

In [20]:
data.to_csv('logi2.csv',index = False)

In [21]:
d = pd.read_csv('logi2.csv')
d.head()

,Id,Expected
0,1,3.058027
1,2,2.958056
2,3,4.842742
3,4,4.779539
4,5,4.291294
